In [2]:
import numpy as np 
import matplotlib.pyplot as plt
from biosurf_cpu import *
import matplotlib
from glob import glob
from scipy.signal import savgol_filter
import os
import warnings; warnings.simplefilter('ignore')
from scipy.signal import butter,filtfilt

cmap = matplotlib.cm.get_cmap('viridis')

def getsurf(file):
    """Returns a numpy array that is the z-values on each point"""
    myh5 = datx2py(file)
    zdata = myh5['Data']['Surface']
    zdata = list(zdata.values())[0]
    zvals = zdata['vals']
    zvals[zvals == zdata['attrs']['No Data']] = np.nan
    zmean = np.nanmean(zvals)
    return zvals

def smoothen(y):
    yhat = savgol_filter(y, 151,3)
    def nan_helper(y):
        return np.isnan(y), lambda z: z.nonzero()[0]
    nans, x= nan_helper(yhat)
    yhat[nans]= np.interp(x(nans), x(~nans), y[~nans])
    return yhat

def find_edge(y, v):
    g2 = np.abs(np.gradient(y))
    #g2 = np.abs(np.gradient(g))
    idx = np.argwhere(g2>v)
    l = idx[0][0]
    h = idx[-1][0]
    #l = np.argmax(g2[:int(len(g2)/2)])
    #h = np.argmax(g2[int(len(g2)/2):])+int(len(g2)/2)
    return l,h

def getcleansurf(img):
    trad = np.nanmean(img, axis=0)
    suma = np.nansum(img, axis=0)
    counts = np.sum(~np.isnan(img), axis=0)
    counts[counts<40] = 0
    f = suma/counts
    f[np.isinf(f)] = np.nan
    return f

def getnewvals(f, l, h, ol, oh):
    lowind = np.max(np.argwhere(~np.isinf(f[:l+ol])))
    highind = np.min(np.argwhere(~np.isinf(f[h-oh:])))
    return lowind, highind+h-(oh-1)

def subline(f, offx, offy):
    xchange = -len(f)
    ychange = (f[0]-offx)-(f[-1]-offy)
    xarray = np.arange(len(f))
    yarray = straightline(xarray, ychange/xchange, (f[0]-offx))
    res = (f-yarray)
    return res

def subquad(f, offx, offy, offc):
    h = len(f)
    l = 0
    center = int((h-l)/2)
    y_low = f[0]-offx
    y_center = f[center]-offc
    y_high = f[-1]-offy
    Y = [y_low, y_center, y_high]
    X = [0, center, len(f)]
    pfit = np.polyfit(X, Y, 2)
    xarray = np.arange(len(f))
    yarray = pfit[0]*xarray**2 + pfit[1]*xarray*1 + pfit[2]
    res = (f-yarray)
    return res
  
def latres(filepath):
    file = datx2py(filepath)
    k1 = list(file['Attributes'].keys())[1]
    k2 = list(file['Attributes'][k1]['attrs'].keys())[2]
    return file['Attributes'][k1]['attrs'][k2]


ModuleNotFoundError: No module named 'biosurf_cpu'

In [3]:
ls

data/              height_changes.jl        README.md
db_creation.jl     height_saturation.ipynb  shape_fitting.ipynb
diffeqflux_fit.jl  LICENSE                  spatial_growth.jl
droplet.ipynb      model_fitting.jl         surface_functions.jl
figs/              models_comparison.ipynb  two_interface.jl
